# Collecting the tweets

## First to connect 

In [ ]:
#We first run the Connecting notebook in order to make the connection with MySQL and Twitter

%run Libraries.ipynb
%run Connecting.ipynb

## Streaming tweets with Tweepy

In [ ]:
#The downloading procedure

class MyStreamListener(tweepy.StreamListener):
    
    def on_status(self, status):
        
        #Avoid retweeted info
      
        if((not status.retweeted) and ('RT @' not in status.text)):
        
            #Extract info from tweets

            id_str = status.id_str
            created_at = status.created_at

            #Clean the text from emojis

            text = deEmojify(status.text) 

            sentiment = TextBlob(text).sentiment
            polarity = sentiment.polarity

            user_location = deEmojify(status.user.location)

            print("\n")
            print(status.text)

            #Store the data collected into the database and the table the user has previously selected

            if db.is_connected():

                cursor = db.cursor()

                query = "INSERT INTO {} (id_str, created_at, text, polarity, user_location) VALUES (%s, %s, %s, %s, %s)".format(table_name)
                val = (id_str, created_at, text, polarity, user_location)

                cursor.execute(query, val)
                db.commit()
                cursor.close()
    
    def on_error(self, status_code):

        #Return false to stop the streaming
        
        if(status_code == 420):
            
            return False

In [ ]:
#The function for removing ascii characters

def deEmojify(text):

    if text:
        
        return text.encode('ascii', 'ignore').decode('ascii')
    
    else:
        
        return None

## Collecting the tweets based on specific track words

In [ ]:
#Stream tweets only when the user has selected to create a new database and a new table or a new table in an existing database.
    #In case the user chooses an existing database and table we fetch the data that is already collected there.

if(answer_mysql == "1"):
    
    if(answer_db == "1" or answer_table == "1" or answer_collecting == '1'):
    
        track_words = ['vaccine']

        myStreamListener = MyStreamListener()
        myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
        myStream.filter(languages = ["en"],track = track_words)